In [7]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import QControlBox
from pytket.extensions.qiskit import AerBackend
from pytket.circuit import CircBox

In [8]:
#Return Inverse Quantum Fourier Transform Circuit
def iqft():
    iqft2 = Circuit(2)
    iqft2.H(0)
    iqft2.CU1(-0.5, 1, 0)
    iqft2.H(1)
    iqft2_box: CircBox = CircBox(iqft2)
    return iqft2_box

In [9]:
def QPE(U):
    QPE = Circuit()
    m = QPE.add_q_register("m", 2)
    s = QPE.add_q_register("p", 1)
    QPE.H(m[0])
    QPE.H(m[1])
    QPE.X(s[0])
    controlled_u_gate = QControlBox(CircBox(U), 1)
    QPE.add_qcontrolbox(controlled_u_gate, [m[1], s[0]])
    QPE.add_qcontrolbox(controlled_u_gate, [m[0], s[0]])
    QPE.add_qcontrolbox(controlled_u_gate, [m[0], s[0]])
    QPE.add_circbox(iqft(), m)
    QPE.measure_register(m, "c")
    return QPE

In [10]:
#Quantum Phase Estimation Circuit
input_angle = 0.49
unitary_circuit = Circuit(1).U1(input_angle, 0)
unitary_circuit.name = "U"
QPE = QPE(unitary_circuit)

In [12]:
backend = AerBackend()
compiled_circ = backend.get_compiled_circuit(QPE)
n_shots = 1000
result = backend.run_circuit(compiled_circ, n_shots)
dir_dict = {(0,0):'right', (1,0):'up', (0,1):'left', (1,1):'down'}
print(result.get_counts(), dir_dict[result.get_counts().most_common(1)[0][0]])

Counter({(1, 0): 998, (0, 1): 1, (1, 1): 1}) up


In [118]:
render_circuit_jupyter(QPE)